# Graph exploration and sampling

Here we practice network exploration with the main graph samplers that can be found in the literature. The principle is the following: The initial graph is too large to be handled and we need to extract a representative part of it for analysis. We hope this reduced subgraph is representative of the large one, and indeed there are theoretical garantees about that for each method. The samplers are designed to preserve particular graph properties when subsampling. We will see that these properties may or may not be relevant for the task we want to perform and some sampler will be more adapted to specific tasks.

In [ ]:
import networkx as nx
import littleballoffur as lbof
import matplotlib.pyplot as plt
from collections import Counter

The main samplers are coded in the Python module called "little ball of fur" https://github.com/benedekrozemberczki/littleballoffur and we will use it here (`pip install littleballoffur`).
The documentation can be found here:
* https://little-ball-of-fur.readthedocs.io

Let us load one of the datasets available in the module.

In [ ]:
# load a graph
#reader = lbof.GraphReader("facebook")
reader = lbof.GraphReader("github")
G = reader.get_graph()
print('Number of nodes: {}, number of edges: {}.'.format(G.number_of_nodes(),G.number_of_edges()))

Let us suppose this graph is too big for our analysis. We need to get a reduced version of it. We can define the size of this reduced dataset.

In [ ]:
# number of nodes in the subgraph
number_of_nodes = int(0.01*G.number_of_nodes())
print('Number of nodes in the subgraph:',number_of_nodes)

There exist several ways for sampling a graph. When you have access to the full graph, you may sample at random edges or nodes, this is the first family. Alternatively, you can start from an initial group of nodes and collect  a part of the graph by exploring (following connections) from them. We shall focus on these latter approaches.

## Exploring the network
For the applications we have in mind, we want to start the exploration from an initial set of nodes. For example, it could be a particular user or group of users in a social network that are posting about a topic we are interested in. We want to know more about this topic and related topics appearing in the exchanges. Our goal is to explore the network around this initial group. Hence we plan to use the exploration methods.

In the following, we experiment the exploration methods on toy graphs. The goal is to understand the different possibility to explore a network, the different parameters and their impact on the sampled network. The initial group of nodes is chosen at random within the exploration functions of `littleballoffur`. So we do not focus on a specific region of the network but rather on the way the exploration is performed. Later on, we will choose initial nodes and apply the exploration to real networks.

We save the graphs in `gexf` file in order to visualize them with Gephi.

In [ ]:
# Metropolis Hasting random walk sampler
sampler = lbof.MetropolisHastingsRandomWalkSampler(number_of_nodes = number_of_nodes)
GMH = sampler.sample(G)
nx.write_gexf(GMH, 'data/gmh.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GMH.number_of_nodes(),GMH.number_of_edges()))

In [ ]:
# Forest Fire sampler
sampler = lbof.ForestFireSampler(number_of_nodes = number_of_nodes)
GFF = sampler.sample(G)
nx.write_gexf(GFF, 'data/gff.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GFF.number_of_nodes(),GFF.number_of_edges()))

A more general and flexible exploration approach called "Spikyball" ([paper](https://www.mdpi.com/1999-4893/13/11/275)) can be used.

In [ ]:
# Fireball sampler is similar to the Forest Fire sampler
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.8, mode='fireball', 
                                initial_nodes_ratio=0.001)
GFB = sampler.sample(G)
nx.write_gexf(GFB, 'data/gfb.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GFB.number_of_nodes(), GFB.number_of_edges()))

In [ ]:
# Coreball sampler
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.1, mode='coreball',
                                initial_nodes_ratio=0.001)
GCB = sampler.sample(G)
# Remove isolated nodes
GCB = nx.Graph(GCB)
GCB.remove_nodes_from(list(nx.isolates(GCB)))
nx.write_gexf(GCB, 'data/gcb.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GCB.number_of_nodes(), GCB.number_of_edges()))

In [ ]:
# Coreball sampler
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.1, mode='coreball',
                                initial_nodes_ratio=0.001, distrib_coeff=2)
GCB2 = sampler.sample(G)
# Remove isolated nodes
GCB2 = nx.Graph(GCB2)
GCB2.remove_nodes_from(list(nx.isolates(GCB2)))
nx.write_gexf(GCB2, 'data/gcb2.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GCB2.number_of_nodes(), GCB2.number_of_edges()))

### Visualisation of the graphs (from Gephi)
Metropolis-Hasting RW, Forest Fire, Fireball and Coreball 
<table><tr>
    <td><img src="data/gmh.png" alt="MetropolisHasting" width="200"> </td>
    <td><img src="data/gff.png" alt="Forest Fire" width="200"></td>
    <td><img src="data/gfb.png" alt="Fireball" width="200"></td>
    <td><img src="data/gcb.png" alt="Coreball" width="200"></td>
</tr></table>


### Degree distribution
Let us see what is the degree distribution of these networks.

In [ ]:
# A function to plot the degree distribution of a given graph
def plot_degree(G,glabel):
    m=1 # minimal degree to display
    degree_freq = nx.degree_histogram(G)
    degrees = range(len(degree_freq))
    plt.scatter(degrees[m:], degree_freq[m:],label=glabel)
    return max(degrees),max(degree_freq)

In [ ]:
plt.figure(figsize=(12, 8)) 
mx1,my1 = plot_degree(GMH,'MH')
mx2,my2 = plot_degree(GFF,'FF')
mx3,my3 = plot_degree(GFB,'FB')
mx4,my4 = plot_degree(GCB,'CB')
mx5,my5 = plot_degree(GCB2,'CB2')

plt.xlim(1, max([mx1,mx2,mx3,mx4,mx5]))
plt.ylim(1, max([my1,my2,my3,my4,my5]))
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend()
plt.show()

### Results for different paramters
We can play with the parameters, for example the sampling probability.

In [ ]:
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.05, mode='fireball', 
                                initial_nodes_ratio=0.001)
GFB = sampler.sample(G)
Gcc = sorted(nx.connected_components(GFB), key=len, reverse=True)
GFB = GFB.subgraph(Gcc[0])
nx.write_gexf(GFB, 'data/gfb2.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GFB.number_of_nodes(), GFB.number_of_edges()))

In [ ]:
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.9, mode='coreball',
                                initial_nodes_ratio=0.001)
GCB = sampler.sample(G)
GCB = nx.Graph(GCB)
Gcc = sorted(nx.connected_components(GCB), key=len, reverse=True)
GCB = GCB.subgraph(Gcc[0])
nx.write_gexf(GCB, 'data/gcb2.gexf')
print('Subgraph with {} nodes and {} edges.'.format(GCB.number_of_nodes(), GCB.number_of_edges()))

### A larger subgraph
We increase the size of the sampled subgraph to have better statistics on the degree distribution.

In [ ]:
number_of_nodes = 3000
# MHRW
sampler = lbof.MetropolisHastingsRandomWalkSampler(number_of_nodes = number_of_nodes)
GMH = sampler.sample(G)
print('MHRW subgraph with {} nodes and {} edges.'.format(GMH.number_of_nodes(),GMH.number_of_edges()))

# FF
sampler = lbof.ForestFireSampler(number_of_nodes = number_of_nodes)
GFF = sampler.sample(G)
print('FF subgraph with {} nodes and {} edges.'.format(GFF.number_of_nodes(),GFF.number_of_edges()))

# Fireball
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.05, mode='fireball', 
                                initial_nodes_ratio=0.001)
GFB = sampler.sample(G)
print('FB subgraph with {} nodes and {} edges.'.format(GFB.number_of_nodes(), GFB.number_of_edges()))

# Coreball
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.1, mode='coreball',
                                initial_nodes_ratio=0.001)
GCB = sampler.sample(G)
print('CB subgraph with {} nodes and {} edges.'.format(GCB.number_of_nodes(), GCB.number_of_edges()))

# Coreball 2
sampler = lbof.SpikyBallSampler(number_of_nodes = number_of_nodes, sampling_probability=0.1, mode='coreball',
                                initial_nodes_ratio=0.001, distrib_coeff=2)
GCB2 = sampler.sample(G)
print('CB2 subgraph with {} nodes and {} edges.'.format(GCB2.number_of_nodes(), GCB2.number_of_edges()))

# Plot degree distribution
plt.figure(figsize=(12, 8)) 
mx1,my1 = plot_degree(GMH,'MH')
mx2,my2 = plot_degree(GFF,'FF')
mx3,my3 = plot_degree(GFB,'FB')
mx4,my4 = plot_degree(GCB,'CB')
mx5,my5 = plot_degree(GCB2,'CB2')

plt.xlim(1, max([mx1,mx2,mx3,mx4,mx5]))
plt.ylim(1, max([my1,my2,my3,my4,my5]))
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend()
plt.show()

### Degrees in the initial graph
How do the different method perform with respect to node degrees in the initial graph? Do they collect more nodes with a high degree or not?

In [ ]:
# Add degree as a node property in the initial graph
# Then we can collect it easily in the usbsampled graph
nx.set_node_attributes(G,dict(G.degree()), name='degree')

In [ ]:
# A function to plot the degree distribution of a given graph
def plot_d_init(G,subgraph,glabel):
    m=1 # minimal degree to display
    d = [G.nodes[i]['degree'] for i in subgraph.nodes()]
    counter_dic = Counter(d)
    degrees = list(counter_dic.keys())
    degree_freq = list(counter_dic.values())
    #degree_freq, degrees = np.histogram(d,bins=50)
    #print(len(degree_freq),len(degrees))
    #degree_freq = nx.degree_histogram(G)
    #degrees = range(len(degree_freq))
    plt.scatter(degrees[m:], degree_freq[m:],label=glabel)
    return max(degrees),max(degree_freq)

In [ ]:
plt.figure(figsize=(12, 8)) 
mx1,my1 = plot_d_init(G,GMH,'MH')
mx2,my2 = plot_d_init(G,GFF,'FF')
mx3,my3 = plot_d_init(G,GFB,'FB')
mx4,my4 = plot_d_init(G,GCB,'CB')
mx5,my5 = plot_d_init(G,GCB2,'CB2')

plt.xlim(1, max([mx1,mx2,mx3,mx4,mx5]))
plt.ylim(1, max([my1,my2,my3,my4,my5]))
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## Exercise
Redo the experiments with different parameters for the fireball and coreball and a different graph.
What are the best exploration methods
* for collecting more high degree nodes? 
* for keeping the same degree distribution? 
* for exploring a larger part of the network?